In [6]:
!pip install boto3 pandas openpyxl 

  Using cached jmespath-0.10.0-py2.py3-none-any.whl (24 kB)
  Using cached s3transfer-0.5.0-py3-none-any.whl (79 kB)


In [7]:
import boto3
import pandas as pd
import openpyxl
import asyncio
import concurrent.futures

In [8]:
def profile_iterator(func):
    for profile in boto3.session.Session().available_profiles:
        try:
            result = func(profile)
        except Exception as e:
            print(e)
        # break
    yield result

In [9]:
def test_iterator(func):
    for profile in boto3.session.Session().available_profiles:
        try:
            result = func(profile)
        except Exception as e:
            print(e)
        break
    yield result

In [10]:
@profile_iterator
def list_s3(profile):
    print("**********************************" + profile)
    default_session=boto3.session.Session(profile_name=profile)
    s3_re=default_session.resource(service_name="s3", region_name="ap-northeast-2")
    #s3_re = boto3.resource('s3')
    # Print out bucket names
    data = []
    for bucket in s3_re.buckets.all():
        print(bucket.name)
        data.append(bucket.name)
    return data

In [22]:
for item in list_s3:
    print(item)

**********************************default
lazysbucket
['lazysbucket']


In [19]:
@test_iterator
def list_ec2(profile):
    print("**********************************" + profile)
    default_session=boto3.session.Session(profile_name=profile)
    ec2_re=default_session.resource(service_name="ec2", region_name="ap-northeast-2")
    #s3_re = boto3.resource('s3')
    # Print out bucket names
    data = []
    for instance in ec2_re.instances.all():
        instance_info = []
        instance_info.append(instance.instance_id)
        instance_info.append(instance.instance_type)
        instance_info.append(instance.state['Name'])
        # instance_info.append(instance.state_reason)
        instance_info.append(instance.platform if instance.platform is not None else "Linux")
        instance_info.append(instance.vpc)
        instance_info.append(instance.subnet)
        # instance_info.append(instance.vpc_id)
        # instance_info.append(instance.subnet_id)
        instance_info.append(instance.public_ip_address)
        instance_info.append(instance.private_ip_address)
        instance_info.append(instance.tags)
        instance_info.append(instance.security_groups)
        instance_info.append(instance.volumes)
        data.append(instance_info)
    print([name['Value'] for name in data[0][8] if name['Key'] == 'Name'][0])
    return data

In [20]:
for item in list_ec2:
    print(item)

**********************************default
blog-prd
[['i-0e9f485ce932f9b1b', 't3.small', 'running', 'Linux', ec2.Vpc(id='vpc-673fbe0c'), ec2.Subnet(id='subnet-4b73c120'), '3.36.32.139', '172.31.9.230', [{'Key': 'Name', 'Value': 'blog-prd'}], [{'GroupName': 'default', 'GroupId': 'sg-32a23448'}], ec2.Instance.volumesCollectionManager(ec2.Instance(id='i-0e9f485ce932f9b1b'), ec2.Volume)]]


In [32]:
@test_iterator
def list_rds(profile):
    print("**********************************" + profile)
    default_session=boto3.session.Session(profile_name=profile)
    client = default_session.client(service_name="rds", region_name="ap-northeast-2")
    paginator = client.get_paginator('describe_db_instances')
    data = []
    for instance in paginator.paginate():
        instance_info = []
        instance_info.append(instance['DBInstances'])
        instance['DBInstances']
        data.append(instance_info)
    return data

In [33]:
for item in list_rds:
    print(item)

**********************************default
[[[{'DBInstanceIdentifier': 'regionmgration', 'DBInstanceClass': 'db.t2.micro', 'Engine': 'mysql', 'DBInstanceStatus': 'available', 'MasterUsername': 'admin', 'Endpoint': {'Address': 'regionmgration.cf6sgf5yuw1p.ap-northeast-2.rds.amazonaws.com', 'Port': 3306, 'HostedZoneId': 'ZLA2NUCOLGUUR'}, 'AllocatedStorage': 20, 'InstanceCreateTime': datetime.datetime(2021, 6, 9, 16, 9, 31, 368000, tzinfo=tzutc()), 'PreferredBackupWindow': '15:46-16:16', 'BackupRetentionPeriod': 0, 'DBSecurityGroups': [], 'VpcSecurityGroups': [{'VpcSecurityGroupId': 'sg-08b5fe21faaa494dd', 'Status': 'active'}], 'DBParameterGroups': [{'DBParameterGroupName': 'default.mysql8.0', 'ParameterApplyStatus': 'in-sync'}], 'AvailabilityZone': 'ap-northeast-2a', 'DBSubnetGroup': {'DBSubnetGroupName': 'default-vpc-673fbe0c', 'DBSubnetGroupDescription': 'Created from the RDS Management Console', 'VpcId': 'vpc-673fbe0c', 'SubnetGroupStatus': 'Complete', 'Subnets': [{'SubnetIdentifier': 